# ClusterAI 2020
# Ciencia de Datos - Ingeniería Industrial - UTN BA
# clase_07: Practica Clustering: Wine Data Set
### Elaborado por: Nicolás Aguirre

# Importamos Librerias

In [ ]:
# importamos las librerías necesarias para trabajar.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn import preprocessing

In [ ]:
# Importamos librerias de PCA y silhouette_score
from sklearn.decomposition import PCA
from  sklearn.metrics import silhouette_score

In [ ]:
# Importamos librerias de Clustering
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage 
from sklearn.cluster import AgglomerativeClustering

# Repositorio del Dataset
## Wine Data Set:

https://archive.ics.uci.edu/ml/datasets/Wine

# Importamos Dataset

In [ ]:
names_col = ['G','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols',
             'Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue',
             'OD280/OD315 of diluted wines','Proline']

wine_df = pd.read_csv('../../datasets/vinos/clusterai2020_clase07_wine.data', delimiter=',', names=names_col)

In [ ]:
wine_df.head()

In [ ]:
wine_df.shape

In [ ]:
wine_df.isnull().sum()

In [ ]:
# Definimos nuestro X
x = wine_df.iloc[:,1:]
display(x.head())
print(x.shape)

In [ ]:
# Vamos a crear un dataframe para guardar los resultados
results_df = pd.DataFrame(columns=['Cluster','Acc','Sill'])

# Auto-Scaling

In [ ]:
# Realizamos un autoscaling con los datos, para todas las features
scaler = preprocessing.StandardScaler().fit(x)

In [ ]:
xscal = scaler.transform(x)

# K-MEANS

In [ ]:
dist_cent = []
sill_list = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=1).fit(xscal)
    sill_score = silhouette_score(xscal,kmeans.labels_)
    sill_list.append(sill_score)
    dist_cent.append(kmeans.inertia_)
plt.plot(range(2, 10), dist_cent, marker='s');
plt.xlabel('N° K')
plt.ylabel('Sum of squared distances')
plt.show()
plt.plot(range(2, 10), sill_list, marker='s');
plt.xlabel('N° K')
plt.ylabel('Silhouette')
plt.show()


In [ ]:
# Generamos un PCA con los datos luego del autoscaling
import matplotlib.cm as cm
from sklearn import decomposition

# Usamos el objeto PCA de la libreria 'decomposition'
# Definimos la cantidad de componentes
n_comps = 13
components = range(1,n_comps + 1) 
pca = decomposition.PCA(n_components=n_comps)

# Ajustamos
pca.fit(xscal)

# Transformamos
xpca = pca.transform(xscal)

# Porcentaje de la varianza explicada por cada PC
eigenvalues = pca.explained_variance_ratio_

# Suma acumulada
eigenvalues_acum = pca.explained_variance_ratio_.cumsum() 

# Graficamos
# Eje Izquierdo
fig, ax1 = plt.subplots(figsize=(9,6))
ax1.set_xlabel('Top PC')
ax1.set_ylabel('Varianza Explicada', color='k')
ax1.bar(components, eigenvalues, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Eje derecho
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Varianza Acumulada', color='k') 
ax2.plot(components, eigenvalues_acum, color='red') 
ax2.tick_params(axis='y', labelcolor='red')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# De la libreria:
# 'Principal axes in feature space, representing the directions of maximum variance in the data'
# The components are sorted by explained_variance_
pd.DataFrame(pca.components_[0:n_comps,:],columns=x.columns)

# En criollo:
# Es la direccion de los ejes de cada componente (autovectores)

In [ ]:
# scatter plot de los datos, solamente con 2 PC
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1])    
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Figura de PC1 y PC2')
plt.show()

In [ ]:
# Generamos un modelo de K means con 3 clusters con los datos autoscalados
kmeans = KMeans(n_clusters=3, random_state=10).fit(xscal)

**Pregunta: Como se define el numero de cluster del Kmean?**

In [ ]:
# una vez realizado el clusering, observamos los labels de cluster asignadas a cada muestra
kmeans.labels_ = kmeans.labels_+1
kmeans.labels_

In [ ]:
# visualizamos los centroides finales de cada cluster
centers = kmeans.cluster_centers_
centers

In [ ]:
# Scatter plot de muestras y centroides con 2 PC: segun Cluster verdadero vs Clustering con K-Means

#Verdadero
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1],c=wine_df['G'].astype(float))    
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering Verdadero')
# K-Means
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1],c=kmeans.labels_.astype(float))
plt.scatter(centers[:,0], centers[:,1], marker="x", color='r',s=150)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering K-Means')
plt.show()

**PREGUNTA:**

**¿Son correctos los centroides?**

### Metricas del Kmeans

In [ ]:
# Metricas
# Cantidad de errores
q_wrong = kmeans.labels_ - wine_df['G']
q_wrong = (q_wrong!=0).sum()
print(q_wrong)
#Accuracy
acc = ( x.shape[0] - q_wrong ) / x.shape[0]
#Sill. score
sill_score = silhouette_score(xscal,kmeans.labels_)
print(acc)
print(sill_score)

In [ ]:
#Guardamos los resultados
results_df = results_df.append({'Cluster':'Kmeans',
                                'Acc':acc,
                                'Sill':sill_score},ignore_index=True)


# Reduccion de Dimensionalidad (RD) + K-Means

Ahora vamos dejar de usar las 13 variables originales.

Vamos a clusterizar con lo que nos puedan explicar UNICAMENTE los primeros 2 PC, y compararemos los resultados.

In [ ]:
reduced_dim = 2
xpca_rd = xpca[:,0:reduced_dim]

In [ ]:
# Generamos el modelo K-means + RD
kmeans_rd = KMeans(n_clusters=3, random_state=10).fit(xpca_rd)

In [ ]:
# una vez realizado el clusering, observamos las labels de cluster asignadas a cada muestra
kmeans_rd.labels_ = kmeans_rd.labels_ +1
kmeans_rd.labels_

In [ ]:
# visualizamos los centroides finales de cada cluster
centers_rd = kmeans_rd.cluster_centers_
centers_rd

In [ ]:
# Scatter plot con 2 PC: segun Cluster verdadero vs Clustering con K-Means+RD

#Verdadero
plt.figure(figsize=(9,6))
plt.scatter(xpca[:,0],xpca[:,1],c=wine_df['G'].astype(float))    
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering Verdadero')
# K-Means + RD
plt.figure(figsize=(9,6))
plt.scatter(xpca_rd[:,0],xpca_rd[:,1],c=kmeans_rd.labels_.astype(float))
plt.scatter(centers_rd[:,0], centers_rd[:,1], marker="x", color='r',s=150)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Clustering K-means+RD')
plt.show()

**PREGUNTA:**

**¿Y aca que sucedió?**

### Metricas RD + K-Means

In [ ]:
# Metricas
q_wrong = kmeans_rd.labels_ - wine_df['G']
q_wrong = (q_wrong!=0).sum()
print(q_wrong)
acc_rd = ( x.shape[0] - q_wrong ) / x.shape[0]

sill_score_rd = silhouette_score(xpca_rd,kmeans_rd.labels_)

print(acc_rd)
print(sill_score_rd)

In [ ]:
#Guardamos los resultados
results_df = results_df.append({'Cluster':'PCA + Kmeans',
                                'Acc':acc_rd,
                                'Sill':sill_score_rd},ignore_index=True)

In [ ]:
pd.DataFrame(pca.components_[0:reduced_dim,:],columns=x.columns)

## Recontrusccion

**Si quisieramos volver a nuestro espacio original, consideremos que:**

X = PCAvalores * PCAdirecciones

X = PCAscores * PCAcomponents 

X = Autovalores * Autovectores

Solo nos quedaria un detalle. Sumar la media de X.

**¿POR QUE?**

X = Autovalores * Autovectores + Xmedia

**Entonces:**

X_reconstruida = Autovalores(n_componentes) * Autovectores(n_componentes) + Xmedia

In [ ]:
mu = np.mean(x, axis=0)
# 2 PCA
xpca_rd = xpca[:,0:reduced_dim]
x_reconstruido = np.dot(xpca_rd, pca.components_[0:reduced_dim,:])
x_reconstruido += mu
x_reconstruido_df = pd.DataFrame(x_reconstruido,columns=x.columns)

# Muestramos la reconstruccion
sample = np.random.randint(0,high=x.shape[0])
display(x.iloc[sample,:].to_frame().transpose())
display(x_reconstruido_df.iloc[sample,:].to_frame().transpose())

## Consultas?

# Clustering Jerárquico

In [ ]:
# Definimos el linkage
Z = linkage(xscal, 'ward')

In [ ]:
# Threshold (Similaridad)
dist_cluster = 10

In [ ]:
plt.figure(figsize=(25, 10))
dendrogram(Z,color_threshold=dist_cluster)
plt.axhline(c='k',linestyle='--', y=dist_cluster)
plt.show()

In [ ]:
#dist_cluster = ?
#q_clusters = ?

dist_cluster = 20
plt.figure(figsize=(25, 10))
dendrogram(Z,color_threshold=dist_cluster)
plt.axhline(c='k',linestyle='--', y=dist_cluster)
plt.show()

cluster = AgglomerativeClustering(n_clusters=None, distance_threshold =dist_cluster,
                                  affinity='euclidean',
                                  linkage='ward')
cluster.fit_predict(xscal)
cluster.labels_= cluster.labels_+1
cluster.labels_

### SOLO PORQUE SABEMOS EL GROUND TRUE DE LOS CLUSTERS LOS AJUSTO MANUALENTE PARA CALCULAR EL ACCURACY ! !

In [ ]:
for i in np.arange(0,x.shape[0]):
    if cluster.labels_[i]==3:
        cluster.labels_[i]=1
    elif cluster.labels_[i]==2:
        cluster.labels_[i]=3
    elif cluster.labels_[i]==1:
        cluster.labels_[i]=2

In [ ]:
cluster.labels_

In [ ]:
plt.figure(figsize=(9, 6))
plt.scatter(xpca[:,0], xpca[:,1], c=cluster.labels_,)
plt.show()

In [ ]:
# Metricas
q_wrong = cluster.labels_ - wine_df['G']
q_wrong = (q_wrong!=0).sum()
print(q_wrong)
acc3 = ( x.shape[0] - q_wrong ) / x.shape[0]
sill_score_h = silhouette_score(xscal,cluster.labels_)

print(acc3)
print(sill_score_h)

In [ ]:
#Guardamos los resultados
results_df = results_df.append({'Cluster':'Hierarchy',
                                'Acc':acc3,
                                'Sill':sill_score_h},ignore_index=True)

# Resultados

In [ ]:
display(results_df)

# Preguntas ?